## HA2

Task1

In [ ]:
pip install bs4

In [1]:
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
from bs4 import BeautifulSoup as bs

In [168]:
# ввод вакансии
#vacancy = input('Какую вакансию вы ищите: ')
vacancy = 'маркетолог' #заглушка, чтобы не вбивать каждый раз текст
vacancy = ' '.join(vacancy.split())
vacancy = vacancy.replace(" ", "+")

# ввод количества страниц
#pages = int(input('Введите количество страниц: '))
pages = 5 #заглушка, чтобы не вбивать каждый раз текст
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
          'Accept' : '*/*'}

vacancy_name = []
link = []
company = []
city = []
pay = []
pay_min = []
pay_max = []
currency = []

for page in range(pages):
    
    url = f'https://hh.ru/search/vacancy?clusters=true&ored_clusters=true&enable_snippets=true&salary=&st=searchVacancy&text={vacancy}&from=suggest_post&page={page}'
    response = requests.get(url, headers = headers)
    
    if response.ok:
    
        soup = bs(response.text, 'html.parser')

        # получаем название вакансию и ссылку на нее
        names = soup.find_all('div', attrs = {'class': 'vacancy-serp-item__info'})
        for name in [name for name in names if len(names) > 0]:
            tag = 'data-qa'
            target = 'vacancy-serp__vacancy-title'
            if name.find('a', attrs = {tag: target}) != None:           
                vacancy_name.append(name.find('a', attrs = {tag: target}).getText())
                link.append('https://hh.ru' + name.find('a', attrs = {tag: target})['href'].split("hh.ru", 1)[1].split("?from=vacancy_search_list", 1)[0]) 

        # получаем название компании
        names = soup.find_all('div', attrs = {'class': 'vacancy-serp-item__meta-info-company'})
        for name in [name for name in names if len(names) > 0]:
            tag = 'data-qa'
            target = 'vacancy-serp__vacancy-employer'
            if name.find('a', attrs = {tag: target}) != None:           
                company.append(name.find('a', attrs = {tag: target}).getText().replace("\xa0", " "))

        # получаем название города
        names = soup.find_all('span', attrs = {'data-qa': 'vacancy-serp__vacancy-address'})
        for name in [name for name in names if len(names) > 0]:
            if name != None:           
                city.append(name.find(text=True, recursive=False))

        # мин и макс зарплату
        names = soup.find_all('div', attrs = {'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})

        for name in [name for name in names if len(names) > 0]:
            #print(name)
            tag = 'data-qa'
            target = 'vacancy-serp__vacancy-compensation'
            if name.find('span', attrs = {tag: target}) != None:
                pay_data = name.find('span', attrs = {tag: target}).getText().replace("\u202f", "").replace("\xa0", "")
                pay.append(pay_data)
                if pay_data.split()[0] == 'от':
                    pay_min.append(pay_data.split()[1])
                    pay_max.append(np.nan)
                    currency.append(pay_data.split()[2])
                elif pay_data.split()[0] == 'до':
                    pay_min.append(np.nan)
                    pay_max.append(pay_data.split()[1])
                    currency.append(pay_data.split()[2])
                else:
                    pay_min.append(pay_data.split()[0])
                    pay_max.append(pay_data.split()[2])
                    currency.append(pay_data.split()[3])            
            else:
                pay.append(np.nan)
                pay_min.append(np.nan)
                pay_max.append(np.nan)
                currency.append(np.nan)



In [169]:

df = pd.DataFrame(list(zip(vacancy_name,link, company, city, pay_min, pay_max, currency)), 
                  columns = ['Вакансия', 'Link', 'Компания', 'Город' , 'Мин ЗП' , 'Макс ЗП', 'Валюта'])

df.to_csv('df.csv', index=False, sep= ',',encoding='utf-8-sig')
df.head(10)

,Вакансия,Link,Компания,Город,Мин ЗП,Макс ЗП,Валюта
0,"Маркетолог, графический дизайнер",https://hh.ru/vacancy/47141312,ООО Апекс,Санкт-Петербург,50000,50000,руб.
1,Интернет-маркетолог,https://hh.ru/vacancy/47138077,LINK group,Москва,70000,NaN,руб.
2,Специалист по контекстной рекламе (интернет-ма...,https://hh.ru/vacancy/46301330,ADWIN.agency,Санкт-Петербург,70000,120000,руб.
3,Маркетолог,https://hh.ru/vacancy/47073526,ТОО ALPION,Алматы,200000,400000,KZT
4,Контекстолог/Специалист по контекстной рекламе,https://hh.ru/vacancy/40228924,ООО Финансово-правовой альянс,Санкт-Петербург,80000,NaN,руб.
5,Маркетолог,https://hh.ru/vacancy/47068640,Частная компания Proportunity Management Compa...,Нур-Султан,300000,NaN,KZT
6,Маркетолог,https://hh.ru/vacancy/47079655,ООО ТД Комплект,Минск,1500,NaN,бел.руб.
7,Интернет-маркетолог,https://hh.ru/vacancy/46372351,Zoon.ru,Москва,120000,NaN,руб.
8,Маркетолог,https://hh.ru/vacancy/47089115,ИП Перкун О.А.,Минск,800,1000,USD
9,Маркетолог,https://hh.ru/vacancy/47102163,ТОО NRG LLP,Экибастуз,NaN,NaN,NaN


Task 2